In [8]:
model_name = "ENB2-Aug-Transfer-All_after_epoch_14"

!git clone https://github.com/MockaWolke/Places.git
!mkdir Places/weights

import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import os
os.chdir("Places")
import ds_generator
import models
import plotting


In [21]:
df = pd.read_csv("../../input/view-directions/data.csv")
df

In [150]:
!zip -r final.zip weights

In [26]:
label_dict = {"Forward":0, "Backward":1,"Sideways":2}
batch_size = 64

def get_csv():
    df = pd.read_csv("../../input/view-directions/data.csv")

    train_df = df.loc[df["ds_type"]=="train"]
    val_df = df.loc[df["ds_type"]=="val"]
    test_df = df.loc[df["ds_type"]=="test"]

    return train_df,val_df,test_df

def mapping(path,label):

    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image, channels=3)
    label = tf.one_hot(label,3)
    return image, label

def get_ds(df, kind = "train"): 
    assert kind in ["train","test","val"]

    path = f"../../input/view-directions/View_Directions/Our_Data/{kind}/"
    imgs = df["key"] + ".jpg"

    #assert all([img in os.listdir(path) for img in imgs]), "Immage not found"

    image_paths = path + imgs

    labels = df["view_direction"].map(label_dict)

    ds = tf.data.Dataset.from_tensor_slices((image_paths,labels))

    ds = ds.map(mapping)
    ds = ds.batch(batch_size=batch_size, drop_remainder=True)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    return ds

def get_train_and_val():

    train_df,val_df,test_df = get_csv()

    train_ds = get_ds(train_df,kind="train")
    val_ds = get_ds(val_df,kind="val")

    return train_ds,val_ds

train_ds, val_ds = get_train_and_val()

In [38]:
model = models.transfer_model("EN2-e14")
model.load_weights("../../input/model-weights/saves_after_e10")
model = models.unfreeze_model(model)

In [40]:
model_loc = "weights/"+ model_name
log_dir = model_loc + "/logs"
checkpoint_path = model_loc + "/saves/weights"
last_model = model_loc + "/last_model"

if model_name not in os.listdir("weights/"):

  os.makedirs(model_loc )
  os.makedirs(log_dir)
  os.makedirs(checkpoint_path)
  os.makedirs(last_model)
  


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.02,
    patience=4,
    verbose=1,
    mode='auto',
    restore_best_weights=True
) 

In [54]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

In [55]:
hist = model.fit(train_ds, epochs=23, validation_data=val_ds,callbacks=[tensorboard_callback,cp_callback,early_stopping_callback],initial_epoch=17)

In [56]:
plotting.plot_hist(hist)

In [58]:
train_df,val_df,test_df = get_csv()

test_ds = get_ds(test_df,kind="test")

model.evaluate(test_ds)

In [60]:
preds = model.predict(test_ds)

In [108]:
list_of_labels = []
for (x,y) in tfds.as_numpy(test_ds):
    
    list_of_labels.append(y)

labels = np.concatenate(list_of_labels,axis=0)

labels = tf.argmax(labels, axis=1)
preds_indices = tf.argmax(preds,axis=1)
confusion_mtx = tf.math.confusion_matrix(labels,preds_indices)

In [156]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))

names = list(label_dict.keys())

sns.heatmap(confusion_mtx, xticklabels=names, yticklabels=names, 
          annot=True, fmt='g')
plt.title("Confusion Matrix of Test Dataset",fontsize=18)
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.savefig("confusion_matrix.jpg")
plt.show()

In [98]:
imgs = []
for i in tfds.as_numpy(test_ds.take(1)):
    
    imgs = i[0]

In [ ]:
yticklabels

In [113]:
preds_indices = preds_indices.numpy()
labels = labels.numpy()

In [151]:
n_row, n_col = 6,6

n_imgs = n_row * n_col

get_names = dict(zip(label_dict.values(),label_dict.keys()))

_, axs = plt.subplots(n_row, n_col, figsize=(12, 12))
axs = axs.flatten()

for i in range(n_imgs):
    ax = axs[i]
    
    l = get_names[labels[i]]
    p = get_names[preds_indices[i]]
    color = "green" if l==p else "red"
    
    ax.imshow(imgs[i])
    ax.axis("off")
    ax.set_title(f"{l}, ({p})",color=color)
    
plt.suptitle("Predictions on Test Data Set, Acc = 95.7%",fontsize=20)
plt.tight_layout()
plt.savefig("Predictions.jpg",dpi=1500)
plt.show()

In [5]:
model.load_weights(checkpoint_path)

In [51]:
ls weights/ENB2-Aug-Transfer-All_after_epoch_14/saves/

In [52]:
model.load_weights("weights/ENB2-Aug-Transfer-All_after_epoch_14/saves/weights")

In [45]:
last_model

In [128]:
model.save('weights/ENB2-Aug-Transfer-All_after_epoch_14/final_model')

In [135]:
old_model = tf.keras.models.load_model('weights/ENB2-Aug-Transfer-All_after_epoch_14/final_model')
old_model.summary()

In [139]:
import tensorflow.keras.layers as layers
def with_out_aug(name=None):

    inputs = tf.keras.Input([260,260,3])

    model = tf.keras.applications.efficientnet.EfficientNetB2(
        include_top=False,
        weights='imagenet',
        input_tensor= inputs)

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(3, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name=name)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    return model

model = with_out_aug(name="final_model")

In [140]:
model.summary()

In [145]:
old_names = [layer.name for layer in old_model.layers[1:]]
new_names = [layer.name for layer in model.layers]

len(old_names)==len(new_names)

In [146]:
for old,new in zip(old_names,new_names):
    print(new)
    model.get_layer(new).set_weights(old_model.get_layer(old).get_weights())

In [147]:
model.evaluate(test_ds)

In [149]:
# excact the same results -> no difference
model.save('weights/ENB2-Aug-Transfer-All_after_epoch_14/final_model_with_out_aug')

In [7]:
test_ds = ds_generator.get_test_ds()